In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
def get_applnews(board,total_page):
    docs = []
    headers = {
      "cookie": "__cfduid=d5f2274637142c63b85585c60c8fa1e9b1502083011; _ga=GA1.2.2014264793.1502083012; over18=1; _gid=GA1.2.504188955.1532926138",\
      "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"\
    }
    domain = 'https://tw.appledaily.com'
    url = '%s/%s/realtime'%(domain,board)
    print(url)
    res =requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text,'lxml')
    for page in range(1,total_page+1,1):
        page_url = '{}/{}/realtime/{}'.format(domain,board,page)
        print(page_url)
        urls =get_applnews_article_list(page_url,headers)
        for url in urls:
            doc = get_applnews_article(url,headers)
            docs.append(doc)
    df = pd.DataFrame(list(filter(None,docs)))
    return df

In [3]:
def get_applnews_article_list(page_url,headers):
    urls = []
    res = requests.get(page_url,headers=headers)
    soup = BeautifulSoup(res.text,'lxml')
    article_list = soup.select('.thoracis .abdominis .rtddd a')
    for article in article_list:
        url = article['href']
#         print(url)
        urls.append(url)
       
    return urls

In [4]:
def get_applnews_article(url,headers):
    try:
        print('--->url: %s'%(url))
        doc = dict()
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.text,'lxml')
 #       author = meta_values[0].text
 #       board = meta_values[1].text
        title = soup.select('.ndArticle .ndArticle_leftColumn h1')[0].text
        dt = soup.select('.ndArticle .ndArticle_leftColumn .ndArticle_creat')[0].text[5:]
        tm = datetime.strptime(dt,'%Y/%m/%d %H:%M').timestamp() 
        content = soup.select('.ndArticle_leftColumn .ndArticle_contentBox .ndArticle_content .ndArticle_margin p')[0].text
  #      doc['author'] = author
  #      doc['board'] = board
        doc['title'] = title
        doc['tm'] = tm
        doc['content'] = content
        doc['url'] = url
    except Exception as e:
        print('%s has error '%(url))
        print(e)
    return doc

In [8]:
board = 'new'
total_page = 1
data = get_applnews(board,total_page)

https://tw.appledaily.com/new/realtime
https://tw.appledaily.com/new/realtime/1
--->url: https://tw.entertainment.appledaily.com/realtime/20180920/1433178/
--->url: https://tw.finance.appledaily.com/realtime/20180920/1433662/
--->url: https://tw.news.appledaily.com/life/realtime/20180920/1433626/
--->url: https://tw.news.appledaily.com/life/realtime/20180920/1432787/
--->url: https://tw.news.appledaily.com/politics/realtime/20180920/1433268/
--->url: https://tw.sports.appledaily.com/realtime/20180920/1433668/
--->url: https://tw.entertainment.appledaily.com/realtime/20180920/1433163/
--->url: https://tw.news.appledaily.com/international/realtime/20180920/1433665/
--->url: https://tw.finance.appledaily.com/realtime/20180920/1433476/
--->url: https://tw.entertainment.appledaily.com/realtime/20180920/1429926/
--->url: https://tw.news.appledaily.com/local/realtime/20180920/1431693/
--->url: https://tw.news.appledaily.com/local/realtime/20180920/1433659/
--->url: https://tw.finance.appledai

In [6]:
data[['author','location']] = data.content.str.extract('（(\w+)[/／](\w+)')

In [7]:
data

,content,title,tm,url,author,location
0,演出《延禧攻略》富察皇后的中國女星秦嵐今早才一身駝色長褲再披長大衣俐落賞Max Maxa 2...,【米蘭時裝周】富察皇后11月襲台有望 賞FENDI秀俏麗,1.537443e+09,https://tw.entertainment.appledaily.com/realti...,邱瓈瑩,米蘭報導
1,GoPro粉絲苦等1年，運動攝影機Hero7照片正式曝光！3C網站B&H Photo搶先披露...,學哀鳳出3機！GoPro最新Hero7曝光 防手震超強,1.537443e+09,https://tw.finance.appledaily.com/realtime/201...,NaN,NaN
2,今天是入秋以來首波境內空氣污染範圍最大的一天，環保署空氣品質監測網顯示，包括彰化縣的線西、彰...,彰化今天灰濛濛 環保局啟動防空污措施,1.537443e+09,https://tw.news.appledaily.com/life/realtime/2...,鄧惠珍,彰化報導
3,【廣編特輯】中秋耍新梗 木曜4爆料烤肉要醃味根據木曜4超玩從月球傳來的消息指出，中秋烤肉，一...,【特企】醃再烤 木曜4×牛頭牌 月球夯肉大公開,1.537402e+09,https://tw.news.appledaily.com/life/realtime/2...,NaN,NaN
4,中信兄弟儘管在9月7日~15日曾佔據戰績首位一周，但其實自進入9月份開始，團隊打擊就嚴重下滑...,中信9月團隊攻擊嚴重下滑 上壘率不到3成,1.537443e+09,https://tw.sports.appledaily.com/realtime/2018...,NaN,NaN
5,(新增動新聞)周遊與老公李朝永將在10月8日補辦婚禮，前天一同出席關懷演藝人員中秋節餐會，同...,陳松勇酸她老公「考古學家」 周遊氣得想告,1.537443e+09,https://tw.entertainment.appledaily.com/realti...,即時新聞中心,綜合報導
6,印度捷特航空（Jet Airways）一班航機上的機組人員忘記打開機艙的增壓系統，無法調節氣...,忘記調節機艙氣壓 客機逾30乘客耳鼻流血,1.537443e+09,https://tw.news.appledaily.com/international/r...,國際中心,綜合外電報導
7,,南山人壽勞資紛爭 金管會將啟動金檢,1.537443e+09,https://tw.finance.appledaily.com/realtime/201...,NaN,NaN
8,,【大平台】今晚8點！Melody崩潰挑鞋包第3彈 直播時尚媽咪「念經」,1.537443e+09,https://tw.entertainment.appledaily.com/realti...,NaN,NaN
9,曾創下許多第一的翁啟惠，並非從小就是資優生。小時候不喜歡念書，功課普通的他，開始發憤讀書，竟...,【蘋果人物3】家裡沒牛可放只好讀書 翁啟惠「永遠挑戰不可能」,1.537443e+09,https://tw.news.appledaily.com/local/realtime/...,NaN,NaN
